### 1. Currying
### 2. Monads
### 3. Memoization

## 1.

#### Lets take a simple function
Most functions, even simple ones, take multiple arguments

In [17]:
def add(a, b, c):
    
    return a + b + c

print(add(10,100, 1000))

1110


### Binding function arguments
- But we can bind arguments to a function, so that we end up with a function that takes one argument less than the original function. This is done with functools.partial()

In [4]:
from functools import partial

def add(a, b, c):
    
    return a + b + c

add_10 = partial(add, 10)
add_10_100 = partial(add_10, 100)
print(add_10_100(1000))

1110


### Currying
- Currying is a specific kind of argument binding, in which we create a sequence of functions that each take exactly one argument. 
- In Python, you can implement this elegantly with a decorator

In [9]:
from inspect import signature

def curry(fnc):
    
    def inner(arg):
        
        if len(signature(fnc).parameters) == 1:
            return fnc(arg)
        return curry(partial(fnc, arg))
    
    return inner

@curry
def add(a, b, c):
    
    return a + b + c

add_10 = add(10)
print(add_10)
add_10_100 = add_10(100)
print(add_10_100(1000))
print(add_10_100)

<function curry.<locals>.inner at 0x000001606D39F620>
1110
<function curry.<locals>.inner at 0x000001606D39F7B8>


## 2.

### Monads

- nan -- (not a number)
    - Is a special numeric value Any operation on **nan** returns **nan**
    - **nan** × 1 = **nan**
    - **nan** overrides operations
- Maybe  -- (Just or Nothing)
    - The **Maybe monad** is like **nan**
    - Two kinds of values
        - Normal values ( Just)
        - Invalid values ( Nothing)
    - Any function applied to Nothing returns Nothing
- List  -- [1, 2, 3]
    - The **List monad** defines a series of values
    - Any function applied to a List monad , Is applied to each element
    - The result is a new List monad

Let's start with some functions,camelcase(), which transform " a string Like this" into"AStringLikeThis"

In [11]:
def camelcase(s):
    
    return ''.join([w.capitalize() for x in s.split('_')])

print(camelcase('some_function'))

SomeFunction


### The Maybe monad
The Maybe monad consists of two kinds of data, which are typically called Just and Nothing. They both behave very simply:
- When a function is bound to a Just value the function is simply executed and the result is stored in a new Just value
- When a function is bound to a Nothing value, the function is bypassed and Nothing is returned right away.
- In addition, when a function generates an error, it returns a Nothing value
See how similar this is to nan behavior?

In [ ]:
class Just:
    
    def __init__(self, value):
        
        self._value = value
        
    def bind(self, fnc):
        
        try:
            return Just(fnc(self._value))
        except:
            return Nothing()
    def __repr__(self):
        
        return self._value


class Nothing:
    
    def bind(self, fnc):
        
        return Nothing()
    
    def __repr__(self):
        
        return 'Nothing'


print(Just('some_function').bind(camelcase))
print(Nothing().bind(camelcase))
print(Just(10).bind(camelcase))

### The List monad
The List monad stores a list of values. When it is bound to a function each value is passed onto the function separately, and the result is stored as another List

In [19]:
class List:
    
    def __init__(self, values):
        
        self._values = values
    
    def bind(self, fnc):
        
        return List([fnc(value) for value in self._values])
    
    def __repr__(self):
        
        return str(self._values)
    
    
List(['some_text', 'more_texxt']).bind(camelcase)

['SomeText', 'MoreTexxt']

## 3.

### Memoization  --  (Remember me!)
- Referential transparency
- Return values depend only on arguments
- Once a return value has been determined
- A function never needs to be called again
- That's memoization!

- all (iterable)
    - Return True if all elements of the iterable are true (or if the iterable is empty). 

##### Aome expensive function
- Let's consider a function that can take a long time to execute

In [32]:
import time

def prime(n):
    
    for i in range(n, 0, -1):
        if all([i // x != i / x for x in range(i-1, 1, -1)]):
            print(i)
            return i

def timer(func, arg):
    
    t0 = time.time()
    func(arg)
    t1 = time.time()
    
    return  t1-t0

print('Took {} s'.format(timer(prime, 1000000)))
print('Took {} s'.format(timer(prime, 1000000)))
print('Took {} s'.format(timer(prime, 99999)))
print('Took {} s'.format(timer(prime, 99999)))

999983
Took 2.5840938091278076 s
999983
Took 2.3856167793273926 s
99991
Took 0.0937502384185791 s
99991
Took 0.09075593948364258 s


#### Caching 
- We can speed up function calls by storing the result in a cache

In [37]:
cache = {}

def cache_prime(n):
    
    if n in cache:
        return cache[n]
    for i in range(n, 0, -1):
        if all([i // x != i / x for x in range(i-1, 1, -1)]):
            cache[n] = i
            return i
        
        
print('Took {} s'.format(timer(cache_prime, 1000000)))
print('Took {} s'.format(timer(cache_prime, 1000000)))
print('Took {} s'.format(timer(cache_prime, 99999)))
print('Took {} s'.format(timer(cache_prime, 99999)))

Took 2.5083281993865967 s
Took 0.0 s
Took 0.10275721549987793 s
Took 0.0 s


#### Memoization
- Memoization is a type of caching in which return values are stored for specific arguments. Therefore, the implementation above is an example of memoization. But it can be implemented more elegantly using a decorator!

In [38]:
def memoize(fnc):
    
    cache = {}
    
    def inner(*args):
        
        if args in cache:
            return cache[args]
        cache[args] = fnc(*args)
        return cache[args]
    
    return inner


@memoize
def memoized_prime(n):

    for i in range(n, 0, -1):
        if all([i // x != i / x for x in range(i-1, 1, -1)]):
            return i

        
print('Took {} s'.format(timer(memoized_prime, 1000000)))
print('Took {} s'.format(timer(memoized_prime, 1000000)))
print('Took {} s'.format(timer(memoized_prime, 99999)))
print('Took {} s'.format(timer(memoized_prime, 99999)))    

Took 2.606009006500244 s
Took 0.0 s
Took 0.10073041915893555 s
Took 0.0 s
